## 데이터 병렬화
- AI모델 하나를 여러 장치에 복사해서 **각자 다른 데이터를 학습하게 하는 방법**

### 쉽게 설명하면 ...
1. 같은 AI 모델을 여러 장치에 복사
2. 전체 데이터를 나눠서 각 장치에 조금씩(batch) 준다
3. 각 장치는 **자기 데이터로 학습**해서, 변화량(gradient)을 계산
4. 마지막엔 **변화량을 평균**내서, 모든 장치의 모델을 **같은 상태**로 맞춘다.

### 🎯 Colab에서 안 되는가?
- DataParallel은 2개 이상의 GPU가 있어야 동작해요.

- Colab 무료 버전이나 일반 Pro 버전에서는 보통 GPU 한 개만 제공돼요.

- torch.cuda.device_count() 해보면 대부분 1이 나옵니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [8]:
print("GPU 개수:", torch.cuda.device_count())

GPU 개수: 1


In [2]:
# 1. 데이터 전처리 및 로딩
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='.\data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

100%|██████████| 170M/170M [00:05<00:00, 33.0MB/s]


- 이미지를 Tensor로 바꾸고,
- 픽셀 값을 -1 ~ 1 범위로 정규화(normalize) 해요.
- CIFAR-10 훈련 데이터를 다운로드하고,
- trainloader를 통해 데이터를 128개씩 묶어서(batch) 불러올 수 있게 해요

In [3]:
# 2. 가단한 CNN ahepf wjddml (ResNet18 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(num_classes=10)

- 사용할 장치: GPU(cuda) 또는 CPU 자동 선택
- 사전 정의된 ResNet-18 모델을 불러오고, 출력 클래스를 CIFAR-10에 맞게 10개로 설정

In [4]:
# 3. DataParallel로 여러 GPU 사용 설정
if torch.cuda.device_count() > 1 :
  print(f"사용 가능한 GPU 수: {torch.cuda.device_count()}개")
  model = nn.DataParallel(model)


model = model.to(device)

- GPU가 2개 이상이면, DataParallel로 감싸서 병렬 처리 가능하게 설정
(자동으로 모델을 여러 GPU에 복사하고 병렬 학습하게 됨)
- 모델을 GPU 또는 CPU로 이동

In [5]:
# 4. 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

- CrossEntropyLoss: 분류 문제에서 자주 쓰는 손실 함수

- SGD: 확률적 경사 하강법
→ 학습률은 0.01, momentum은 관성값(학습 안정성에 도움)

In [6]:
# 5. 학습 루프
for epoch in range(5) : # 에폭 5회 (전체 데이터를 5번 반복해서 학습)
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0) : # trainloader로부터 한 배치씩 불러옴
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device) # 데이터를 GPU로 이동

    # 기울기 초기화
    optimizer.zero_grad()

    # forward + backward + optimaze
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # 통계 출력
    running_loss += loss.item()
    if i%100 == 99 : # 매 100 미니배치마다 출력
      print(f"[Epoch {epoch + 1}, Step {i + 1}] Loss: {running_loss / 100:.3f}")
      running_loss = 0.0


print("학습 완료")


[Epoch 1, Step 100] Loss: 1.889
[Epoch 1, Step 200] Loss: 1.529
[Epoch 1, Step 300] Loss: 1.379
[Epoch 2, Step 100] Loss: 1.132
[Epoch 2, Step 200] Loss: 1.098
[Epoch 2, Step 300] Loss: 1.064
[Epoch 3, Step 100] Loss: 0.888
[Epoch 3, Step 200] Loss: 0.896
[Epoch 3, Step 300] Loss: 0.862
[Epoch 4, Step 100] Loss: 0.715
[Epoch 4, Step 200] Loss: 0.763
[Epoch 4, Step 300] Loss: 0.740
[Epoch 5, Step 100] Loss: 0.555
[Epoch 5, Step 200] Loss: 0.599
[Epoch 5, Step 300] Loss: 0.636
학습 완료


- 기울기 초기화 → 예측 → 손실 계산 → 역전파 → 파라미터 업데이트

- 이 과정은 각 GPU가 자신의 데이터로 따로 실행되고,
DataParallel이 기울기를 평균내서 모든 GPU 모델을 동기화해줍니다!
- 매 100번째 배치마다 **손실(loss)**을 출력해요. 학습 상황을 확인할 수 있어요.

## 📌 핵심 요약
- DataParallel은 모델을 복제해서 데이터 병렬화를 자동으로 처리합니다.

- 각 GPU는 서로 다른 데이터(batch)를 처리하고,
기울기를 평균 내서 모델을 동기화합니다.

- 구현이 아주 간단해서 실험용 코드에 자주 사용돼요.

